In [1]:
import functools

import numpy as np
import pennylane as qml
from scipy.linalg import expm

from graddft_qnn.dft_qnn import DFTQNN
from graddft_qnn.unitary_rep import O_h

# Defining

In [2]:
unitary_reps = [O_h._180_deg_rot()]

In [15]:
unitary_reps

[array([[0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.]])]

In [3]:
XII = functools.reduce(np.kron, [qml.X.compute_matrix(), np.eye(2), np.eye(2)])
IXI = functools.reduce(np.kron, [np.eye(2), qml.X.compute_matrix(), np.eye(2)])
IIX = functools.reduce(np.kron, [np.eye(2), np.eye(2), qml.X.compute_matrix()])

In [4]:
YII = functools.reduce(np.kron, [qml.Y.compute_matrix(), np.eye(2), np.eye(2)])
IYI = functools.reduce(np.kron, [np.eye(2), qml.Y.compute_matrix(), np.eye(2)])
IIY = functools.reduce(np.kron, [np.eye(2), np.eye(2), qml.Y.compute_matrix()])

In [5]:
ZII = functools.reduce(np.kron, [qml.Z.compute_matrix(), np.eye(2), np.eye(2)])
IZI = functools.reduce(np.kron, [np.eye(2), qml.Z.compute_matrix(), np.eye(2)])
IIZ = functools.reduce(np.kron, [np.eye(2), np.eye(2), qml.Z.compute_matrix()])

$$
ZZ(\phi) = \exp\left(-i \frac{\phi}{2} (Z \otimes Z)\right) =
\begin{bmatrix}
    e^{-i \phi / 2} & 0 & 0 & 0 \\
    0 & e^{i \phi / 2} & 0 & 0 \\
    0 & 0 & e^{i \phi / 2} & 0 \\
    0 & 0 & 0 & e^{-i \phi / 2}
\end{bmatrix}
$$

$ZZ(0) = I$

In [21]:
ZZZ_gen = -1j * np.pi / 3 * functools.reduce(np.kron, [qml.Z.compute_matrix(), qml.Z.compute_matrix(), qml.Z.compute_matrix()])

In [22]:
ZZZ = expm(ZZZ_gen)

In [7]:
def process(gate_matrix, u_reprs: list[np.array]):
    gate_matrix = gate_matrix.astype(np.complex64)
    gen = DFTQNN.twirling(gate_matrix, unitary_reps=u_reprs)
    gate = expm(-1j * gen)
    return qml.pauli_decompose(
        gate, check_hermitian=False, hide_identity=True, pauli=True
    )

# Run the twirling + generator

In [8]:
process(XII, unitary_reps), process(IXI, unitary_reps), process(IIX, unitary_reps)

((0.5403022766113281+0j) * I
 + -0.8414709568023682j * X(0),
 (0.5403022766113281+0j) * I
 + -0.8414709568023682j * X(1),
 (0.5403022766113281+0j) * I
 + -0.8414709568023682j * X(2))

In [9]:
process(YII, unitary_reps), process(IYI, unitary_reps), process(IIY, unitary_reps)

((0.5403022766113281+0j) * I
 + 0.8414709568023682j * Y(0),
 (0.5403022766113281+0j) * I
 + 0.8414709568023682j * Y(1),
 (0.5403022766113281+0j) * I
 + (-0-0.8414709568023682j) * Y(2))

In [14]:
process(YII, unitary_reps), process(IYI, unitary_reps), process(IIY, unitary_reps)

((0.5403022766113281+0j) * I
 + 0.8414709568023682j * Y(0),
 (0.5403022766113281+0j) * I
 + 0.8414709568023682j * Y(1),
 (0.5403022766113281+0j) * I
 + (-0-0.8414709568023682j) * Y(2))

In [11]:
process(ZII, unitary_reps), process(IZI, unitary_reps), process(IIZ, unitary_reps)

((0.5403022766113281+0j) * I
 + 0.8414709568023682j * Z(0),
 (0.5403022766113281+0j) * I
 + 0.8414709568023682j * Z(1),
 (0.5403022766113281+0j) * I
 + -0.8414709568023682j * Z(2))

### A funny case

In [17]:
circuit = [
    qml.X.compute_matrix() @ qml.Y.compute_matrix() @ qml.Z.compute_matrix(),
    qml.X.compute_matrix() @ qml.Y.compute_matrix() @ qml.Z.compute_matrix(),
    qml.X.compute_matrix() @ qml.Y.compute_matrix() @ qml.Z.compute_matrix(),
]

In [19]:
circuit_matrix = functools.reduce(np.kron, circuit)

In [20]:
qml.pauli_decompose(
    circuit_matrix, check_hermitian=False, hide_identity=True, pauli=True
)

-1j * I

### Fix the case

By adding $ZZZ$

In [23]:
process(ZZZ, unitary_reps)

(1.2277654856443405-0.6707313656806946j) * I
+ (-0.8586366921663284+0.46907538175582886j) * Z(0) @ Z(1) @ Z(2)

https://quantumcomputing.stackexchange.com/questions/13740/how-to-implement-linear-combination-of-two-unitary-gates-in-a-quantum-circuit